## Setup

Before running this notebook, here are some setup steps:
+ Clone the Clipper repo (`git clone --recursive https://github.com/ucbrise/clipper.git && cd clipper`)
+ Install Anaconda with Python 2
+ Create a new Anaconda environment (`conda create -n ENV_NAME python=2`) and activate it (follow the Anaconda instructions).
+ Install a local version of Clipper admin (`pip install -e ./clipper_admin`)

In [1]:
!pwd

/Users/rahulbalakrishnan/Desktop/clipper/clipper_admin


In [1]:
from clipper_admin import ClipperConnection, KubernetesContainerManager as K8sManager

In [2]:
cm = K8sManager(kubernetes_api_ip="eric-dev.clipper-k8s-dev.com", useInternalIP=True)



In [3]:
cl = ClipperConnection(cm)
# cl.connect()

In [4]:
nr = 2
cl.start_clipper(num_replicas = nr)



18-03-31:04:32:22 INFO     [kubernetes_container_manager.py:172] Found 5 nodes: ec2-54-186-235-230.us-west-2.compute.amazonaws.com, ec2-54-190-21-100.us-west-2.compute.amazonaws.com, ec2-54-149-160-111.us-west-2.compute.amazonaws.com, ec2-54-149-129-227.us-west-2.compute.amazonaws.com, ec2-54-201-98-152.us-west-2.compute.amazonaws.com
18-03-31:04:32:22 INFO     [kubernetes_container_manager.py:180] Setting Clipper mgmt port to 32574
18-03-31:04:32:22 INFO     [kubernetes_container_manager.py:184] Setting Clipper mgv2 port to 32633
18-03-31:04:32:22 INFO     [kubernetes_container_manager.py:191] Setting Clipper query port to 30841
18-03-31:04:32:22 INFO     [clipper_admin.py:113] Clipper still initializing.
18-03-31:04:32:23 INFO     [clipper_admin.py:115] Clipper is running


In [8]:
k8s_v1 = cl.cm._k8s_v1
k8s_beta = cl.cm._k8s_beta
# cm.set_num_replicas("query-frontend", 9033148, None, None, 3)


In [9]:
k8s_v1.read_namespaced_endpoints(name="mgmt-frontend", namespace="default")


{'api_version': 'v1',
 'kind': 'Endpoints',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2018, 3, 31, 11, 32, 22, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'initializers': None,
              'labels': {u'ai.clipper.container.label': '',
                         u'ai.clipper.name': 'mgmt-frontend'},
              'name': 'mgmt-frontend',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '17757762',
              'self_link': '/api/v1/namespaces/default/endpoints/mgmt-frontend',
              'uid': '2d7d0881-34d7-11e8-a8c5-022a73b21532'},
 'subsets': [{'addresses': None,
              'not_ready_addresses': [{'hostname': None,
                                  

In [11]:
endpoints = k8s_v1.read_namespaced_endpoints(name="query-frontend", namespace="default")
query_frontend_ips = [addr.ip for addr in endpoints.subsets[0].addresses]
query_frontend_ips

['100.96.3.138', '100.96.4.138']

In [7]:
ep2 = k8s_v1.read_namespaced_endpoints(name="redis", namespace="default")
qf2 = [addr.ip for addr in ep2.subsets[0].addresses]
qf2

['100.96.4.85']

In [5]:
from clipper_admin import __version__ as v
# from clipper_admin.deployers import python as python_deployer
# python_deployer.deploy_python_closure(cl, name="sum-model", version=1, input_type="doubles", func=feature_sum)
# cl.link_model_to_app(app_name="hello-world", model_name="sum-model")

# from clipper_admin import ClipperConnection, DockerContainerManager as dl
# cl2 = ClipperConnection(dl())

model_name = "test-model1"

cl.build_and_deploy_model(model_name, 1, "ints", "test.txt", "clipper/noop-container:{}".format(v),
                          container_registry = "rbala19", num_replicas = nr)

# cl2.build_model(
#         model_name,
#         v,
#         "test.txt",
#         "clipper/noop-container:{}".format(v),
#         container_registry = "rbala19")

# # cl.build_model(
# #         "test-model",
# #         1,
# #         "test.txt",
# #         "clipper/noop-container:{}".format(v),
# #         container_registry = "rbala19")

# cm.deploy_model(model_name, 
#                 v, 
#                 "ints", 
#                 "clipper/noop-container:{}".format(v)
#                )








18-03-31:04:32:33 INFO     [clipper_admin.py:393] Building model Docker image with model data from test.txt
18-03-31:04:32:33 INFO     [clipper_admin.py:395] <Image: 'test-model:1', 'rbala19/dead-model1:1', 'rbala19/dead-model2:1', 'rbala19/test-model1:1', 'rbala19/test-model1:2', 'rbala19/test-model1:3', 'rbala19/test-model1:4', 'rbala19/test-model1:develop', 'rbala19/test-model2:1', 'rbala19/test-model2:develop', 'rbala19/test-model3:develop', 'rbala19/test-model4:develop', 'rbala19/test-model:1'>
18-03-31:04:32:33 INFO     [clipper_admin.py:397] Pushing model Docker image to rbala19/test-model1:1
18-03-31:04:32:37 INFO     [kubernetes_container_manager.py:300] Finished deploying model
18-03-31:04:32:37 INFO     [kubernetes_container_manager.py:305] Size of ret is 2
18-03-31:04:32:37 INFO     [clipper_admin.py:1101] Posted successfully


ConnectionError: HTTPConnectionPool(host='ec2-54-186-235-230.us-west-2.compute.amazonaws.com', port=32574): Max retries exceeded with url: /admin/add_model (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10409ee90>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [10]:
model_name = "dead-model1"

# cl.build_and_deploy_model(model_name, 1, "ints", "test.txt", "clipper/noop-container:{}".format(v),
#                           container_registry = "rbala19", num_replicas = 1, new_ip = '100.96.4.98')


cl.build_model(model_name, 1, "test.txt", "clipper/noop-container:{}".format(v), container_registry = "rbala19")

cm.deploy_model(model_name, 1, "ints", "clipper/noop-container:{}".format(v), 1, '100.96.3.117')


18-03-27:18:05:51 INFO     [clipper_admin.py:395] Building model Docker image with model data from test.txt
18-03-27:18:05:51 INFO     [clipper_admin.py:397] <Image: 'test-model:1', 'rbala19/dead-model1:1', 'rbala19/dead-model2:1', 'rbala19/test-model1:1', 'rbala19/test-model1:2', 'rbala19/test-model1:3', 'rbala19/test-model1:4', 'rbala19/test-model1:develop', 'rbala19/test-model2:1', 'rbala19/test-model2:develop', 'rbala19/test-model3:develop', 'rbala19/test-model4:develop', 'rbala19/test-model:1'>
18-03-27:18:05:51 INFO     [clipper_admin.py:399] Pushing model Docker image to rbala19/dead-model1:1
Traceback (most recent call last):
  File "clipper_admin/kubernetes/kubernetes_container_manager.py", line 297, in deploy_model
    self.stop_models([original_name])
  File "clipper_admin/kubernetes/kubernetes_container_manager.py", line 422, in stop_models
    for v in models[m]:
TypeError: list indices must be integers, not str
18-03-27:18:05:57 INFO     [kubernetes_container_manager.py:3

In [16]:
import redis
r = redis.Redis()
print(r.get('100.96.3.71'))

None


# Notes

### Considerations when rebalancing model containers
When the number of query frontend replicas changes and we rebalance, we need to tell some of the model containers that the query frontend IP address has changed and they should re-connect to the new address. The easiest way to do so is by simply killing the containers and restarting them. But for containers with a long initialization time, it would potentially be a useful optimization if we could have them reconnect to a new address without restarting. One potential drawback with this is if one of those swapped model containers dies and is re-started by Kubernetes, I believe it will be restarted with the Docker runtime arguments it was originally started with. As a result, this will cause it to attempt to re-connect to the original query-frontend replica which no longer exists.

### Where to store Clipper cluster state?